In [4]:
from huggingface_hub import notebook_login

# Authenticate to Hugging Face in a notebook environment
notebook_login()

In [20]:
from PIL import Image
import io
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
from huggingface_hub import HfApi, HfFolder
from toolbox.printing import debug
import random
CATEGORIES = ["equation", "figure", "table", "plot", "algorithm"]
CAT_INDEX = 3

for category in CATEGORIES[CAT_INDEX:CAT_INDEX+1]:
    df = pd.read_csv(f"dataset_{category}.csv")
    # Shuffle dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    train_df, valid_df = train_test_split(df, test_size=0.2)

    def load_image(image_path):
        if image_path.lower().endswith(".png") or image_path.lower().endswith(".jpg"):
            with open(image_path, "rb") as f:
                return Image.open(io.BytesIO(f.read()))
        else:
            with open(image_path, "rb") as f:
                return io.BytesIO(f.read())

    def read_file(file_path):
        with open(file_path, "r") as file:
            return file.read()

    def transform(row):
        row["tex_code"] = read_file(row["tex_code"])
        for key in ["output"]:
            if row[key] is not None and len(row[key]) > 0:
                row[key] = load_image(row[key])
        return row

    train_dataset = Dataset.from_pandas(train_df).map(transform).shuffle()
    valid_dataset = Dataset.from_pandas(valid_df).map(transform).shuffle()

    from datasets import Features, Value, Image, load_dataset

    # Define the features of your dataset
    features = Features({
        'id': Value('int64'),
        'tex_code': Value('string'),
        'category': Value('string'),
        'subject': Value('string'),
        'output': Image(),        # For image features
        'asset_0': Value('string'),
        'asset_1': Value('string'),
        'asset_2': Value('string'),
        'asset_3': Value('string'),
        'asset_4': Value('string'),
        'asset_5': Value('string'),
        'asset_6': Value('string'),
        'asset_7': Value('string'),
        'asset_8': Value('string'),
        'asset_9': Value('string'),
        '__index_level_0__': Value('int64')
    })

    train_dataset = train_dataset.cast(features)
    valid_dataset = valid_dataset.cast(features)

    dataset_dict = DatasetDict({"train": train_dataset, "validation": valid_dataset})
    # help(dataset_dict.push_to_hub)
    # Then save or upload as before


    dataset_dict.push_to_hub("JosselinSom/Latex-VLM")

Map:   0%|          | 0/382 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/382 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/96 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.72k [00:00<?, ?B/s]